In [1]:
!pip install tensorflow==1.5
!pip install keras

    100% |████████████████████████████████| 44.4MB 18kB/s eta 0:00:011  8% |██▉                             | 4.0MB 1.5MB/s eta 0:00:27    39% |████████████▋                   | 17.6MB 2.9MB/s eta 0:00:10    90% |████████████████████████████▉   | 40.0MB 1.1MB/s eta 0:00:04
    100% |████████████████████████████████| 7.1MB 97kB/s eta 0:00:011
    100% |████████████████████████████████| 13.9MB 55kB/s eta 0:00:01 0% |▏                               | 71kB 211kB/s eta 0:01:06    36% |███████████▌                    | 5.0MB 1.0MB/s eta 0:00:09    43% |█████████████▊                  | 6.0MB 1.3MB/s eta 0:00:07    47% |███████████████▏                | 6.6MB 1.2MB/s eta 0:00:07    88% |████████████████████████████▎   | 12.2MB 1.5MB/s eta 0:00:02
    100% |████████████████████████████████| 92kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 265kB/s ta 0:00:011    58% |██████████████████▉             | 1.8MB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

Using TensorFlow backend.


### data setup

#### load input and output sentance vectors

In [3]:
word_em_in = './data/docspace.tsv'

# load word embeddings
word_embeddings = {}
with open(word_em_in, 'r', encoding='utf-8') as w_in:
    for line in w_in:
        line = line.replace('\n','')
        line = line.split('\t')
        word_embeddings[line[0]] = np.array(line[1:]).astype(float)

print(word_embeddings['hello'].shape)

(100,)


In [4]:
conversation_in = './data/conversation.txt'

conversations_embedding_list_x = []
conversations_embedding_list_y = []
with open(conversation_in, 'r', encoding='utf-8') as c_in:
    for line in c_in:
        line = line.replace('\n','')
        line = line.split('\t')
        utterance_embedding_list = []
        convs_len = len(line)
        if convs_len > 1:
            for utterance in line:
                sentance_embedding = np.zeros(100,)
                wcount = 0.0
                for word in utterance.split(' '):
                    sentance_embedding = np.add(sentance_embedding, word_embeddings[word])
                    wcount = wcount + 1.0
                sentance_embedding = np.divide(sentance_embedding, wcount)
                utterance_embedding_list.append(sentance_embedding)
            conversations_embedding_list_x.append(np.array(utterance_embedding_list[0:convs_len-1]))
            conversations_embedding_list_y.append(np.array(utterance_embedding_list[1:]))

print(len(conversations_embedding_list_x))

83096


In [15]:
conversations_embedding_list_x[4].shape

(4, 100)

In [23]:
print(conversations_embedding_list_x[0])
print(conversations_embedding_list_y[0])

[[ 1.68135413e-03 -5.56713325e-03  5.68291917e-03  1.57534167e-03
   5.47224942e-03 -2.18916038e-03 -5.38892250e-03 -2.26224167e-03
   9.26792833e-04 -9.52086667e-04 -5.47860375e-03 -4.67241667e-05
  -4.89935167e-03 -2.07906333e-03  9.26537875e-04  7.46528608e-03
   3.32129500e-03 -4.50536008e-03 -1.85978967e-03 -3.05233333e-03
  -4.74591492e-03 -1.66448267e-03  9.49448333e-04 -3.21437667e-03
   2.17751333e-03 -3.59858417e-03  5.83542750e-03  4.06461715e-03
  -6.27228583e-03  1.63854384e-03 -1.11029467e-02 -4.81279367e-03
  -5.25145417e-03 -3.01928225e-03  1.80717417e-03  2.02799858e-03
   2.19037167e-03 -3.35099025e-03 -1.35250125e-03  1.30034750e-03
  -3.82643833e-03 -5.94615333e-03  1.32180967e-03 -6.56965333e-04
   2.21355992e-03 -3.82444217e-03  2.13365417e-03 -2.60544142e-03
   1.58487500e-03 -3.32173333e-03 -8.94620346e-03  5.32725800e-03
   8.87144250e-03  2.47404908e-03 -9.39900083e-04  7.34554317e-03
  -7.17393258e-03 -9.42627500e-03  1.98624817e-03 -2.27368042e-03
   1.60264

#### create a generator

In [49]:
from random import randint
data_x_limit = len(conversations_embedding_list_x)
def generator():
    while True:
        index = randint(0, data_x_limit-1)
        yield np.array([conversations_embedding_list_x[index]]), np.array([conversations_embedding_list_y[index]])

### build LSTM model

In [50]:
model = Sequential()
model.add(LSTM((100), batch_input_shape=(None, None, (100)), return_sequences=True))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, None, 100)         80400     
Total params: 80,400
Trainable params: 80,400
Non-trainable params: 0
_________________________________________________________________


In [52]:
history = model.fit_generator(generator(), samples_per_epoch=data_x_limit, nb_epoch=50)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=83096, epochs=50)`
  """Entry point for launching an IPython kernel.


Epoch 1/50
83096/83096 [==============================] - 267s 3ms/step - loss: 4.8454e-05 - acc: 0.0585
Epoch 2/50
83096/83096 [==============================] - 254s 3ms/step - loss: 4.8045e-05 - acc: 0.0650
Epoch 3/50
83096/83096 [==============================] - 250s 3ms/step - loss: 4.7931e-05 - acc: 0.0654
Epoch 4/50
83096/83096 [==============================] - 250s 3ms/step - loss: 4.7570e-05 - acc: 0.0663
Epoch 5/50
83096/83096 [==============================] - 256s 3ms/step - loss: 4.7875e-05 - acc: 0.0685
Epoch 6/50
 3848/83096 [>.............................] - ETA: 4:19 - loss: 4.6652e-05 - acc: 0.0699

KeyboardInterrupt: 

In [46]:
!pip install matplotlib

    100% |████████████████████████████████| 12.6MB 64kB/s eta 0:00:01   13% |████▌                           | 1.8MB 2.8MB/s eta 0:00:04    49% |████████████████                | 6.3MB 1.7MB/s eta 0:00:04    94% |██████████████████████████████▏ | 11.8MB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 1.2MB/s ta 0:00:01
    100% |████████████████████████████████| 952kB 717kB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 1.5MB/s ta 0:00:01
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.show()